In [ ]:
pip install chromadb

In [ ]:
pip install chardet

In [ ]:
import chardet
import json

# Detect file encoding
with open('generated-data.json', 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
    encoding = result['encoding']

In [ ]:
import chromadb
import json

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="new2_collection")

# Step 1: Read the JSON file with explicit encoding
with open('Rawdata.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Step 2: Process the data
documents = []
metadatas = []
ids = []

for item in data:
    # Check if required keys are present in the item
    if 'caption' in item and 'mentioned' in item and 'Plots' in item and 'web location' in item:
        # Concatenate caption and mentioned for the document
        document = f"{item['caption']} {item['mentioned']}"
        documents.append(document)

        # Create metadata with Plots and web location
        metadata = {"mentioned": item["mentioned"], "Plots": item["Plots"], "web_location": item["web location"]}
        metadatas.append(metadata)

        # Use Plots value as the ID
        ids.append(item["Plots"])
    else:
        # Handle the case where required keys are missing for an item
        print(f"Skipping item due to missing keys: {item}")

# Step 3: Use the structured data with collection.add
collection.add(documents=documents, metadatas=metadatas, ids=ids)


In [ ]:
results = collection.query(
    query_texts=["this is our document"],
    n_results=3
)

In [ ]:
print(results)

In [ ]:
import matplotlib.pyplot as plt
import urllib
from PIL import Image

# Extract information from the results dictionary
ids = results['ids'][0]
distances = results['distances'][0]
metadatas = results['metadatas'][0]
web_location = metadatas[0]['web_location']
caption = metadatas[0]['Plots']
paragraphs = metadatas[0]['mentioned']

# Print captions and save URL and captions of paragraphs
print(f"Caption: {caption}")
print(f"Web Location: {web_location}")
print("Paragraphs:")
for paragraph in paragraphs.split('. '):
    print(paragraph)

# Download and display the picture
image_url = web_location
image_path = 'temp_image.png'
urllib.request.urlretrieve(image_url, image_path)
img = Image.open(image_path)
plt.imshow(img)
plt.axis('off')
plt.show()
